In [ ]:
import pandas as pd
import pytz
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('1.xlsx',  index_col=0)
df.head(2)

In [ ]:
# Step 2: Convert `Created (US/Eastern)` to datetime and localize to `US/Eastern`
df['Created (US/Eastern)'] = pd.to_datetime(
    df['Created (US/Eastern)'].str[:-12],  # Remove `US/Eastern` text
    format="%Y-%m-%d %H:%M:%S:%f"
).dt.tz_localize('US/Eastern')  # Add timezone information

In [ ]:
# Step 3: Convert to local time (keeping US/Eastern unchanged)
df['Created (Local)'] = df['Created (US/Eastern)'].dt.tz_convert('US/Eastern')

In [ ]:
# Step 4: Extract the time and date for grouping
df['Created Time'] = df['Created (Local)'].dt.time  # Extract time for daily trends
df['Created Day'] = df['Created (Local)'].dt.date  # Extract date for separate graphs

In [ ]:
# Graph 1: Alerts over 7 days (Days on x-axis and count on y-axis)
alerts_per_day = df.groupby('Created Day').size()

plt.figure(figsize=(10, 6))
alerts_per_day.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title("Alerts Over 7 Days")
plt.xlabel("Date")
plt.ylabel("Number of Alerts")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Graph 2: Alerts per day in separate graphs (Time on x-axis and count on y-axis)
unique_days = df['Created Day'].unique()
fig, axes = plt.subplots(nrows=7, ncols=1, figsize=(10, 30), sharex=True)

for i, day in enumerate(unique_days[:7]):  # Limit to 7 days if more exist
    daily_data = df[df['Created Day'] == day]
    # Count alerts by time for visualization
    time_count = daily_data.groupby('Created Time').size()
    
    # Convert `datetime.time` to strings for plotting
    time_count.index = time_count.index.astype(str)
    
    # Plotting
    axes[i].plot(time_count.index, time_count.values, marker='o', label=str(day))
    axes[i].set_title(f"Alerts on {day}")
    axes[i].set_ylabel("Count of Alerts")
    axes[i].tick_params(axis='x', rotation=45)  # Rotate x-axis labels for readability
    axes[i].grid(True)
    axes[i].legend()

In [ ]:
# Final adjustments to layout
plt.xlabel("Time (US/Eastern)")
plt.tight_layout()
plt.show()